In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
df = pd.read_csv('V1.csv')

how many samples

In [39]:
df.shape[0]

4000

In [40]:
df.shape

(4000, 14)

avg house price

In [41]:
df.PRICE.mean()

np.float64(24.355923220694248)

How many houses have 5 or more rooms?

In [42]:
(df.RM>=5).sum()


np.int64(3953)

In [43]:
# df.PRICE.nlargest(10).mean()

#or
df.PRICE.sort_values(ascending=False).iloc[:10].mean()

np.float64(52.36590175716407)

What is the total number of missing or unknown values in the number of rooms feature?

In [44]:
df.RM.unique()
(df.RM == -1).sum()

np.int64(40)

In [45]:
df.AGE.unique()
(df.AGE<0).sum()

np.int64(50)

In [46]:
df.RIVERSIDE.unique()
(df.RIVERSIDE == 'UNKNOWN').sum()

np.int64(88)

In [47]:
df[(df.RIVERSIDE == 'YES')&(df.AGE<=50)&(df.AGE>=0)].shape

(44, 14)

In [48]:
df.HIGHWAYCOUNT.isin([6,7,8]).sum()

np.int64(1211)

 Create a column 'CATEGORY' and divide the houses in categories as following:
Category 1: house price <10 lacs
Category 2: 10 lacs <= house price <20 lacs
Category 3: 20 lacs <= house price <30 lacs
Category 4: 30 lacs <= house price <40 lacs
Category 5: house price >=40 lacs
Which category has the highest number of records?

In [49]:
# Solution
def make_cat(price):

  if price<10:
    return "Category 1"

  elif price >= 10 and price < 20:
    return "Category 2"

  elif price >= 20 and price < 30:
    return "Category 3"

  elif price>=30 and price < 40:
    return "Category 4"

  else:
    return "Category 5"


df['Category'] = df.PRICE.apply(make_cat)

In [50]:
df.Category.value_counts().max()

np.int64(2028)

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
df['AGE'].replace({-2:np.nan}, inplace=True)
df['RM'].replace({-1:np.nan}, inplace=True)
df['RIVERSIDE'].replace({'UNKNOWN':np.nan}, inplace=True)
X = df.drop('PRICE', axis=1)
y = df['PRICE']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_9736\982319492.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AGE'].replace({-2:np.nan}, inplace=True)
C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_9736\982319492.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [53]:
X_train.shape[0]

2800

Apply following preprocessing steps:
Drop CATEGORY column
CRIM: min max scaling
ZN: min max scaling
INDUS: standard scaling
POLINDEX: min max scaling
DIS: min max scaling
HIGHWAYCOUNT: min max scaling
TAX: min max scaling
PTRATIO: min max scaling
IMM: min max scaling
BPL: min max scaling
RM: impute with median then min max scaling
AGE: impute with mean then min max scaling
RIVERSIDE: Impute with most frequent value then one hot encode.

In [59]:
X_train = X_train.drop('Category', axis=1)
X_test = X_test.drop('Category', axis=1)



KeyError: "['Category'] not found in axis"

In [ ]:
min_max_cols = X_train.columns.tolist()
min_max_cols.remove['INDUS']
min_max_cols.remove['RIVERSIDE']
min_max_cols

In [60]:
from inspect import CO_ASYNC_GENERATOR
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

imp_ct = ColumnTransformer([
    ('mean', SimpleImputer(strategy='mean'), ['AGE']),
    ('median', SimpleImputer(strategy='median'), ['RM']),
    ('mode', SimpleImputer(strategy='most_frequent'), ['RIVERSIDE']),
], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

ct = ColumnTransformer([
    ('min_max', MinMaxScaler(), min_max_cols),
    ('std', StandardScaler(), ['INDUS']),
    ('ohe', OneHotEncoder(sparse_output=False), ['RIVERSIDE'])
], verbose_feature_names_out=False).set_output(transform='pandas')

NameError: name 'min_max_cols' is not defined

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
# X_train, X_test = X_train.drop('Category', axis=1), X_test.drop('Category', axis=1)


# ct = ColumnTransformer([
#     ('mean', SimpleImputer(missing_values=-2), ['AGE']),
#     ('median', SimpleImputer(missing_values=-1, strategy='median'), ['RM']),
#     ('most_freq', SimpleImputer(missing_values='UNKNOWN', strategy='most_frequent'), ['RIVERSIDE'])
# ], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')


# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)

# ct = ColumnTransformer([
#     ('minmax', MinMaxScaler(), ['CRIM', 'ZN', 'POLINDEX', 'DIS', 'HIGHWAYCOUNT', 'TAX', 'PTRATIO', 'IMM', 'BPL', 'RM', 'AGE']),
#     ('std', StandardScaler(), ['INDUS']),
#     ('RIVERSIDE', OneHotEncoder(sparse_output=False), ['RIVERSIDE'])
# ], remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

# X_train = ct.fit_transform(X_train)
# X_test = ct.transform(X_test)

ValueError: Input X contains NaN.
SimpleImputer does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
print(X_train.columns)
print(X_test.columns)


Index(['CRIM', 'ZN', 'INDUS', 'POLINDEX', 'RM', 'AGE', 'DIS', 'HIGHWAYCOUNT',
       'TAX', 'PTRATIO', 'IMM', 'BPL', 'RIVERSIDE'],
      dtype='object')
Index(['CRIM', 'ZN', 'INDUS', 'POLINDEX', 'RM', 'AGE', 'DIS', 'HIGHWAYCOUNT',
       'TAX', 'PTRATIO', 'IMM', 'BPL', 'RIVERSIDE'],
      dtype='object')
